# Hyperparameter Tuning

This notebook performs hyperparameter tuning for the French Cards Detector model.
We will use the custom training loop defined in `train_custom.py` and vary key parameters to find the best configuration.

In [1]:
import os
import torch
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from train_custom import train

sns.set_style("whitegrid")

## 1. Define Arguments Class
We need a class to mimic `argparse` arguments.

In [2]:
class Args:
    def __init__(self, **kwargs):
        self.data = 'datasets/unified/data.yaml'
        self.model = 'yolo11n.pt'
        self.epochs = 3  # Short epochs for tuning
        self.batch = 16
        self.imgsz = 640
        self.lr = 1e-3
        self.mosaic = 0.5
        self.device = 'mps' if torch.backends.mps.is_available() else 'cpu'
        self.project = 'runs/tuning'
        self.workers = 0
        
        for k, v in kwargs.items():
            setattr(self, k, v)

## 2. Define Search Space
We will tune:
- Learning Rate (`lr`)
- Mosaic Probability (`mosaic`)

In [3]:
learning_rates = [1e-4, 1e-3, 1e-2]
mosaic_probs = [0.0, 0.5, 1.0]

results = []

print(f"Starting tuning with {len(learning_rates) * len(mosaic_probs)} combinations...")

Starting tuning with 9 combinations...


## 3. Run Tuning Loop

In [ ]:
for lr in learning_rates:
    for mosaic in mosaic_probs:
        run_name = f"lr_{lr}_mosaic_{mosaic}"
        print(f"\n--- Running {run_name} ---")
        
        args = Args(
            lr=lr,
            mosaic=mosaic,
            project=f"runs/tuning/{run_name}"
        )
        
        try:
            final_loss = train(args)
            results.append({
                'lr': lr,
                'mosaic': mosaic,
                'val_loss': final_loss
            })
            print(f"Finished {run_name}. Val Loss: {final_loss:.4f}")
        except Exception as e:
            print(f"Failed {run_name}: {e}")
            results.append({
                'lr': lr,
                'mosaic': mosaic,
                'val_loss': float('inf')
            })


--- Running lr_0.0001_mosaic_0.0 ---
Using device: mps
Train images: 64138
Val images: 10735
Loading model: yolo11n.pt


Epoch 1/3:   0%|          | 9/4009 [00:48<4:23:10,  3.95s/it, loss=228]     

Epoch 1/3:   0%|          | 10/4009 [00:53<5:57:29,  5.36s/it, loss=171]


## 4. Analyze Results

In [ ]:
df_results = pd.DataFrame(results)
print(df_results.sort_values('val_loss'))

# Pivot for heatmap
pivot_table = df_results.pivot(index='lr', columns='mosaic', values='val_loss')

plt.figure(figsize=(10, 8))
sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap="viridis_r")
plt.title("Validation Loss by Hyperparameters")
plt.show()

In [ ]:
best_run = df_results.loc[df_results['val_loss'].idxmin()]
print("Best Parameters:")
print(best_run)